In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 97.2 MB/s eta 0:00:00


In [2]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}


def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


predict_step(['10815824_2997e03d76.jpg']) # ['a woman in a hospital bed with a woman in a hospital bed']


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.10.cros

['a man standing next to a brown horse near a fire hydrant']

In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.2 MB/s eta 0:00:00
   

In [ ]:
from transformers import pipeline
import gradio as gr

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

def predict(file_path):
  out = image_to_text(file_path)
  return out[0]['generated_text']

image_input = gr.inputs.Image(type='filepath', label="Image File")
text_output = gr.outputs.Textbox(label="Text Output")

interface = gr.Interface(fn=predict, inputs=image_input, outputs=text_output)
interface.launch(share = True,debug=True)

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.10.cros

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://755dae8b8094709f34.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
